In [1]:
from datasets import load_dataset

passage_dataset = load_dataset("llm-book/aio-passages", split="train")

Generating train split:   0%|          | 0/4288198 [00:00<?, ? examples/s]

In [2]:
print(passage_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'text', 'section', 'title'],
    num_rows: 4288198
})


In [3]:
from pprint import pprint

pprint(passage_dataset[0])
pprint(passage_dataset[1])

{'id': 1,
 'pageid': 5,
 'revid': 88740876,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'id': 2,
 'pageid': 5,
 'revid': 88740876,
 'section': '語源',
 'text': '英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" '
         'も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" '
         'のように唱えられていた。また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。"&" '
         'はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。結果として、アルファベットの復唱の最後は "X, Y, '
         'Z, and per se and" という形になった。この最後のフレーズが繰り返されるうちに "ampersand" '
         'と訛っていき、この言葉は1837年までには英語の一般的な語法となった。',
 'title': 'アンパサンド'}


In [4]:
from transformers import AutoModel, AutoTokenizer

In [5]:
model_name = "llm-book/bert-base-japanese-v3-bpr-passage-aio"
tokenizer = AutoTokenizer.from_pretrained(model_name)
passage_encoder = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [6]:
# GPUのメモリに移動
device = "cuda:0"
passage_encoder = passage_encoder.to(device)

In [7]:
import numpy as np
import torch

In [8]:
def embed_passages(
    titles: list[str],
    texts: list[str],
) -> np.ndarray:
    tokenized = tokenizer(
        titles,
        texts,
        padding=True,
        truncation="only_second",
        max_length=256,
        return_tensors="pt",
    ).to(device)

    with torch.inference_mode():
        with torch.cuda.amp.autocast():
            encoded = passage_encoder(**tokenized).last_hidden_state[:, 0]

    emb = encoded.cpu().numpy()
    print(emb)
    emb = np.where(emb < 0, 0, 1).astype(bool)
    emb = np.packbits(emb).reshape(emb.shape[0], -1)
    print(emb)
    return emb

In [9]:
embed1 = embed_passages(
    [
        passage_dataset[0]["title"],
        passage_dataset[1]["title"],
    ],
    [
        passage_dataset[0]["text"],
        passage_dataset[1]["text"],
    ],
)

[[ 0.07001754 -0.6588069  -0.4974683  ... -0.11826201  0.3735757
  -0.00703129]
 [ 0.3707856  -0.9401323  -0.29569283 ...  0.00737173  0.3040893
  -0.2848628 ]]
[[133 162 145  21 151 215 254 119 214  80   4 189 177  53 100 115  68 177
   70 103  41  90 127 227 113  27  71 148  92 162 176 133 105  99  16  16
   52  70 132  46   2  32 211 149  29 103  53 233  29 199 124  65 178  90
   60  32 201 114 214 132  60 254 216 249 184  57 119 181  23 253 121  83
   63 115 141  73  90   0 239 225 194 248  16 108  66 215 124 248  64  26
  214  78  52 118 115 194]
 [132 147 209   6 147 223 190  99 212  80  38 152 176  53 102 235  21 187
   76  98 207  82  43 123 213  26 231 130  88 226 241  21 249  35  24  48
   49   6   4 174 130  32 247 133 173 113  53 110  57 199 108 213 102  86
   29  43 201 115 215 134  60 254 216 185 176  41 103 245  29 212 121  57
   43 119 172 193  90   2 239 229 194 122  32  96  66 247 244 120   3  58
  244 206  54 112  51  70]]
